# Import Packages

In [25]:
import os
import glob
import tqdm
import warnings

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import statsmodels.api as sm

from statsmodels.tsa.arima.model import ARIMA

warnings.simplefilter("ignore")

# Load Data

In [2]:
data = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1201055 entries, 0 to 1201054
Data columns (total 32 columns):
 #   Column                                   Non-Null Count    Dtype  
---  ------                                   --------------    -----  
 0   Date                                     1201055 non-null  object 
 1   FIPS                                     1201055 non-null  object 
 2   MMWR_week                                1201055 non-null  int64  
 3   Recip_County                             1201055 non-null  object 
 4   Recip_State                              1201055 non-null  object 
 5   Series_Complete_Pop_Pct                  1201055 non-null  float64
 6   Series_Complete_Yes                      1201055 non-null  int64  
 7   Series_Complete_12Plus                   1184588 non-null  float64
 8   Series_Complete_12PlusPop_Pct            1184588 non-null  float64
 9   Series_Complete_18Plus                   1201055 non-null  int64  
 10  Series_Complete_18

In [4]:
data.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,...,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity
0,12/13/2021,12071,50,Lee County,FL,58.4,450202,448034.0,65.5,429908,...,C,12.0,12.0,12.0,12.0,Metro,4.0,4.0,4.0,4.0
1,12/13/2021,21155,50,Marion County,KY,46.9,9040,8951.0,55.2,8457,...,D,15.0,16.0,16.0,16.0,Non-metro,7.0,8.0,8.0,8.0
2,12/13/2021,UNK,50,Unknown County,KY,0.0,143323,142403.0,0.0,136422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/13/2021,22099,50,St. Martin Parish,LA,37.8,20194,20142.0,44.8,18979,...,D,14.0,15.0,15.0,15.0,Metro,2.0,3.0,3.0,3.0
4,12/13/2021,08125,50,Yuma County,CO,40.5,4053,4043.0,49.6,3881,...,C,11.0,11.0,12.0,12.0,Non-metro,7.0,7.0,8.0,8.0


# Data Cleaning

In [5]:
# See all unique FIPS
fips = data['FIPS'].unique()
print(fips)

['12071' '21155' 'UNK' ... 45035 53053 55109]


In [6]:
# Pick only useful columns.
data = data[[
    'Date',
    'FIPS',
    'Recip_County',
    'Recip_State',
    'Series_Complete_Pop_Pct',
    'Series_Complete_Yes',
    'Series_Complete_12Plus',
    'Series_Complete_12PlusPop_Pct',
    'Series_Complete_18Plus',
    'Series_Complete_18PlusPop_Pct',
    'Series_Complete_65Plus',
    'Series_Complete_65PlusPop_Pct',
    'Completeness_pct'
]]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1201055 entries, 0 to 1201054
Data columns (total 13 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   Date                           1201055 non-null  object 
 1   FIPS                           1201055 non-null  object 
 2   Recip_County                   1201055 non-null  object 
 3   Recip_State                    1201055 non-null  object 
 4   Series_Complete_Pop_Pct        1201055 non-null  float64
 5   Series_Complete_Yes            1201055 non-null  int64  
 6   Series_Complete_12Plus         1184588 non-null  float64
 7   Series_Complete_12PlusPop_Pct  1184588 non-null  float64
 8   Series_Complete_18Plus         1201055 non-null  int64  
 9   Series_Complete_18PlusPop_Pct  1201055 non-null  float64
 10  Series_Complete_65Plus         1201055 non-null  int64  
 11  Series_Complete_65PlusPop_Pct  1201055 non-null  float64
 12  Completeness_p

In [7]:
data.head()

,Date,FIPS,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct
0,12/13/2021,12071,Lee County,FL,58.4,450202,448034.0,65.5,429908,67.5,195622,87.1,95.0
1,12/13/2021,21155,Marion County,KY,46.9,9040,8951.0,55.2,8457,57.8,2648,81.1,94.0
2,12/13/2021,UNK,Unknown County,KY,0.0,143323,142403.0,0.0,136422,0.0,37898,0.0,94.0
3,12/13/2021,22099,St. Martin Parish,LA,37.8,20194,20142.0,44.8,18979,46.6,5762,68.6,95.0
4,12/13/2021,08125,Yuma County,CO,40.5,4053,4043.0,49.6,3881,53.4,1327,70.9,95.0


In [8]:
# Let's cast all Series data into float64.
series_columns = [c for c in data.columns if 'Series' in c]
data[series_columns] = data[series_columns].astype(np.float64)
# Convert Date to datetime.
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1201055 entries, 0 to 1201054
Data columns (total 13 columns):
 #   Column                         Non-Null Count    Dtype         
---  ------                         --------------    -----         
 0   Date                           1201055 non-null  datetime64[ns]
 1   FIPS                           1201055 non-null  object        
 2   Recip_County                   1201055 non-null  object        
 3   Recip_State                    1201055 non-null  object        
 4   Series_Complete_Pop_Pct        1201055 non-null  float64       
 5   Series_Complete_Yes            1201055 non-null  float64       
 6   Series_Complete_12Plus         1184588 non-null  float64       
 7   Series_Complete_12PlusPop_Pct  1184588 non-null  float64       
 8   Series_Complete_18Plus         1201055 non-null  float64       
 9   Series_Complete_18PlusPop_Pct  1201055 non-null  float64       
 10  Series_Complete_65Plus         1201055 non-null  float

In [9]:
data.head()

,Date,FIPS,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct
0,2021-12-13,12071,Lee County,FL,58.4,450202.0,448034.0,65.5,429908.0,67.5,195622.0,87.1,95.0
1,2021-12-13,21155,Marion County,KY,46.9,9040.0,8951.0,55.2,8457.0,57.8,2648.0,81.1,94.0
2,2021-12-13,UNK,Unknown County,KY,0.0,143323.0,142403.0,0.0,136422.0,0.0,37898.0,0.0,94.0
3,2021-12-13,22099,St. Martin Parish,LA,37.8,20194.0,20142.0,44.8,18979.0,46.6,5762.0,68.6,95.0
4,2021-12-13,08125,Yuma County,CO,40.5,4053.0,4043.0,49.6,3881.0,53.4,1327.0,70.9,95.0


In [10]:
print(f'Total nmber of rows: {len(data)}')

Total nmber of rows: 1201055


In [11]:
print('Are there any null rows for the essential reference data?')
print(f'Date: {data["Date"].isnull().sum()}')
print(f'Recip_County: {data["Recip_County"].isnull().sum()}')
print(f'Recip_State: {data["Recip_State"].isnull().sum()}')

Are there any null rows for the essential reference data?
Date: 0
Recip_County: 0
Recip_State: 0


Ok. So we are good. Probably only the `Series_Complete_12Plus` columns have null values as we saw in the `info` panel.

In [12]:
data['Series_Complete_12Plus'][data['Series_Complete_12Plus'].isnull()]

102       NaN
167       NaN
190       NaN
205       NaN
320       NaN
           ..
1200529   NaN
1200653   NaN
1200837   NaN
1200847   NaN
1200962   NaN
Name: Series_Complete_12Plus, Length: 16467, dtype: float64

We have to be aware about those `NaN` values when we group the data on states.

# Create Time Series by State

Let's see all the existing states.

In [13]:
states = data['Recip_State'].unique()
states.sort()
print(f'A total of {len(states)} states:')
print(states)

A total of 60 states:
['AK' 'AL' 'AR' 'AS' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'FM' 'GA' 'GU'
 'HI' 'IA' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MH' 'MI' 'MN'
 'MO' 'MP' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK'
 'OR' 'PA' 'PR' 'PW' 'RI' 'SC' 'SD' 'TN' 'TX' 'UNK' 'UT' 'VA' 'VI' 'VT'
 'WA' 'WI' 'WV' 'WY']


In [14]:
counties = data['Recip_County'].unique()
counties.sort()
print(f'A total of {len(counties)} counties:')
print(counties)

A total of 1960 counties:
['Abbeville County' 'Acadia Parish' 'Accomack County' ... 'Zapata County'
 'Zavala County' 'Ziebach County']


In [15]:
# Let's see the Date Range
dates = data['Date'].unique()
num_dates = len(dates)
dates.sort()
print(f'First: {dates[0]}')
print(f'Last: {dates[-1]}')
print(f'A total of: {num_dates}')
print(dates)

First: 2020-12-13T00:00:00.000000000
Last: 2021-12-13T00:00:00.000000000
A total of: 366
['2020-12-13T00:00:00.000000000' '2020-12-14T00:00:00.000000000'
 '2020-12-15T00:00:00.000000000' '2020-12-16T00:00:00.000000000'
 '2020-12-17T00:00:00.000000000' '2020-12-18T00:00:00.000000000'
 '2020-12-19T00:00:00.000000000' '2020-12-20T00:00:00.000000000'
 '2020-12-21T00:00:00.000000000' '2020-12-22T00:00:00.000000000'
 '2020-12-23T00:00:00.000000000' '2020-12-24T00:00:00.000000000'
 '2020-12-25T00:00:00.000000000' '2020-12-26T00:00:00.000000000'
 '2020-12-27T00:00:00.000000000' '2020-12-28T00:00:00.000000000'
 '2020-12-29T00:00:00.000000000' '2020-12-30T00:00:00.000000000'
 '2020-12-31T00:00:00.000000000' '2021-01-01T00:00:00.000000000'
 '2021-01-02T00:00:00.000000000' '2021-01-03T00:00:00.000000000'
 '2021-01-04T00:00:00.000000000' '2021-01-05T00:00:00.000000000'
 '2021-01-06T00:00:00.000000000' '2021-01-07T00:00:00.000000000'
 '2021-01-08T00:00:00.000000000' '2021-01-09T00:00:00.000000000'
 

We can see that they start from the half of december of 2020 to the half of december of 2021 with a frequency of 2 days. So we have 1 year of data.

In [16]:
parsed_groups = dict()
state_groups = data.groupby('Recip_County')
for idx, (group_name, group) in enumerate(state_groups):
    # Set the Date as index & drop duplicates.
    group.set_index('Date', drop=True, inplace=True)
    group = group[~group.index.duplicated(keep='first')]
    
    # Reconstruct the DF with all the possible dates for consistency .
    template_df = pd.DataFrame(index=dates, columns=group.columns)
    template_df.update(group)
    # Fill possible NaN values with the edges.
    group.fillna('bfill', inplace=True)
    group.fillna('ffill', inplace=True)
    
    parsed_groups[group_name] = template_df

# Plot Data

In [17]:
FIPS = {
    'AL':'01',
    'AK':'02',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'FL':'12',
    'GA':'13',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PA':'42',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'WA':'53',
    'WV':'54',
    'WI':'55',
    'WY':'56',
    'AS':'60',
    'GU':'66',
    'MP':'69',
    'PR':'72',
    'VI':'78'
}

In [18]:
data18_65 = {k: v['Series_Complete_18PlusPop_Pct'] for k, v in parsed_groups.items()}
data18_65 = pd.DataFrame(index=dates, columns=list(data18_65.keys()), data=data18_65)
data18_65.head()

,Abbeville County,Acadia Parish,Accomack County,Ada County,Adair County,Adams County,Addison County,Adjuntas Municipio,Aguada Municipio,Aguadilla Municipio,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
2020-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-12-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
data18_65.tail()

,Abbeville County,Acadia Parish,Accomack County,Ada County,Adair County,Adams County,Addison County,Adjuntas Municipio,Aguada Municipio,Aguadilla Municipio,...,Yoakum County,Yolo County,York County,Young County,Yuba County,Yukon-Koyukuk Census Area,Yuma County,Zapata County,Zavala County,Ziebach County
2021-12-09,44.5,63.6,80.2,70.1,56.1,55.5,72.3,76.0,74.8,82.1,...,52.8,72.7,85.6,48.9,60.1,79.1,74.1,74.4,60.3,31.8
2021-12-10,44.5,63.7,80.3,70.2,43.4,63.2,72.3,76.1,74.9,82.1,...,52.8,72.8,85.7,49.0,60.2,79.2,74.2,74.4,60.3,32.0
2021-12-11,44.6,63.7,80.3,70.2,49.1,61.9,72.4,76.1,75.0,82.2,...,53.0,72.9,65.2,49.0,60.2,79.2,53.2,74.5,60.4,32.0
2021-12-12,44.6,63.8,80.5,70.3,56.2,77.6,72.4,76.1,75.0,82.2,...,53.0,72.9,63.2,49.0,60.3,79.2,74.5,74.5,60.4,32.1
2021-12-13,44.6,63.8,80.5,70.3,40.8,63.4,72.5,76.1,75.0,82.2,...,53.0,73.0,54.0,49.1,60.4,79.3,53.4,74.5,60.4,32.1


In [30]:
# Plot the last timestep to see how it looks.
data18_65_to_plot = data18_65.iloc[-1]
fips = [county.iloc[0]['FIPS'] for county in parsed_groups.values()]
fips = [(0 if np.isnan(f) else f) for f in fips]
fig = ff.create_choropleth(
    fips=fips,
    values=data18_65_to_plot.values,
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    state_outline={'color': 'rgb(255,255,255)', 'width': 1},
    simplify_county=0.04,
    simplify_state=0.04,
    legend_title='Density %',
    title='Vaccinated People'
)
fig.show()

In [22]:
# Export vaccination density at all timesteps.
if not os.path.exists('images'):
    os.mkdir('images')

fips = [county.iloc[0]['FIPS'] for county in parsed_groups.values()]
fips = [(0 if np.isnan(f) else f) for f in fips]
for t in tqdm.tqdm(range(len(data18_65))):
    data18_65_to_plot = data18_65.iloc[t]
    fig = ff.create_choropleth(
        fips=fips,
        values=data18_65_to_plot.values,
        county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
        legend_title='Vaccinated People Density'
    )
    fig.write_image(f'images/Series_Complete_18PlusPop_Pct/{t}.png')

In [28]:
# Create a video out of the frames.
random_img = cv2.imread('images/Series_Complete_18PlusPop_Pct/0.png')
frame_height, frame_width, _ = random_img.shape
out = cv2.VideoWriter('Series_Complete_18PlusPop_Pct.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
image_files = glob.glob('images/Series_Complete_18PlusPop_Pct/*.png')
image_files = sorted(image_files,  key=lambda item: int(os.path.splitext(os.path.basename(item))[0]))
for image_file in image_files:
    image = cv2.imread(image_file)
    out.write(image)
out.release()

In [ ]:
# Wrap up all this code into a single function
def create(column: str):
    data = {k: v[column] for k, v in parsed_groups.items()}
    data = pd.DataFrame(index=dates, columns=list(data.keys()), data=data)
    
    # Export vaccination density at all timesteps.
    if not os.path.exists('images'):
        os.mkdir('images')

    fips = [county.iloc[0]['FIPS'] for county in parsed_groups.values()]
    fips = [(0 if np.isnan(f) else f) for f in fips]
    for t in tqdm.tqdm(range(len(data))):
        data_to_plot = data.iloc[t]
        fig = ff.create_choropleth(
            fips=fips,
            values=data.values,
            county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
            legend_title=f'Vaccinated People Density for: {column}'
        )
        fig.write_image(f'images/{column}/{t}.png')
        
    random_img = cv2.imread(f'images/{column}/0.png')
    frame_height, frame_width, _ = random_img.shape
    out = cv2.VideoWriter(f'{column}.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    
    image_files = glob.glob(f'images/{column}/*.png')
    image_files = sorted(image_files,  key=lambda item: int(os.path.splitext(os.path.basename(item))[0]))
    for image_file in image_files:
        image = cv2.imread(image_file)
        out.write(image)
    out.release()

In [ ]:
# Create videos for the other age categories.
create(column='Series_Complete_12PlusPop_Pct')
create(column='Series_Complete_65PlusPop_Pct')

# Let's Make Some Predictions

In [ ]:
# https://machinelearningmastery.com/make-sample-forecasts-arima-python/